# Wildfire Risk - AutoML
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [2]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Checking Pre-Requisites From Previous Notebook

In [3]:
%store -r autopilot_train_s3_wildfire

In [4]:
try:
    autopilot_train_s3_wildfire
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


In [5]:
if not autopilot_train_s3_wildfire:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [6]:
import boto3
import sagemaker
import pandas as pd
import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Training Data

In [7]:
print(autopilot_train_s3_wildfire)

s3://sagemaker-us-east-1-857283526476/data/wildfire_risk_autopilot.csv


In [8]:
!aws s3 ls $autopilot_train_s3_wildfire

2023-04-06 16:45:57    6233547 wildfire_risk_autopilot.csv


In [9]:
#See the prepared training data
!aws s3 cp $autopilot_train_s3_wildfire ./tmp/

download: s3://sagemaker-us-east-1-857283526476/data/wildfire_risk_autopilot.csv to tmp/wildfire_risk_autopilot.csv


In [25]:
import csv

df = pd.read_csv("./tmp/wildfire_risk_autopilot.csv")
df.head()

,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,DT00,...,CARBON_LITTER,CARBON_SOIL_ORG,CARBON_STANDING_DEAD,CARBON_UNDERSTORY_AG,CARBON_UNDERSTORY_BG,STATE,WATERCD,LAT,LON,fire
0,USR0000OKE2,44.6661,-118.9208,1560.6,"KEENEY TWO OREGON, OR US",4.8,U,4.8,",U",0.0,...,14.783938,27.720882,5.667277,1.030544,0.114505,OR,0.0,44.599231,-118.867771,0.0
1,USR0000CBRK,36.0931,-118.2603,2499.4,"BLACKROCK CALIFORNIA, CA US",0.0,U,0.0,",U",0.0,...,16.527200,22.216451,7.324724,0.489455,0.054384,CA,0.0,36.064916,-118.218929,0.0
2,USR0000CBOO,38.9875,-123.3486,196.3,"BOONVILLE CALIFORNIA, CA US",0.0,U,0.0,",U",0.0,...,NaN,NaN,NaN,NaN,NaN,CA,0.0,38.992735,-123.333129,0.0
3,USR0000CKON,38.9119,-122.7064,659.3,"KONOCTI CALIFORNIA, CA US",448.3,U,232.8,",U",0.0,...,14.140916,12.305077,0.000000,1.904834,0.211648,CA,0.0,38.844304,-122.791696,0.0
4,USR0000CWHH,33.9839,-118.0100,289.6,"WHITTIER HILLS CALIFORNIA, CA US",285.3,U,167.7,",U",0.0,...,NaN,NaN,NaN,NaN,NaN,CA,NaN,33.867761,-117.963377,1.0


# Setup the S3 Location for the Autopilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [26]:
prefix_model_output = "models/autopilot"

model_output_s3_wildfire = "s3://{}/{}".format(bucket, prefix_model_output)

print(model_output_s3_wildfire)

s3://sagemaker-us-east-1-857283526476/models/autopilot


In [27]:
max_candidates = 10

job_config1 = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 900,
        "MaxCandidates": max_candidates,
        "MaxAutoMLJobRuntimeInSeconds": 5400,
    },
}

input_data_config1 = [
    {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "{}".format(autopilot_train_s3_wildfire)}},
        "TargetAttributeName": "fire",
    }
]

output_data_config1 = {"S3OutputPath": "{}".format(model_output_s3_wildfire)}

In [28]:
#Check for existing autopilot jobs

existing_jobs_response = sm.list_auto_ml_jobs()
existing_jobs_response

{'AutoMLJobSummaries': [{'AutoMLJobName': 'automl-fire-06-16-52-30',
   'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-fire-06-16-52-30',
   'AutoMLJobStatus': 'InProgress',
   'AutoMLJobSecondaryStatus': 'AnalyzingData',
   'CreationTime': datetime.datetime(2023, 4, 6, 16, 53, 35, 581000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2023, 4, 6, 16, 54, 45, 196000, tzinfo=tzlocal())},
  {'AutoMLJobName': 'automl-dm-30-03-35-20',
   'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-dm-30-03-35-20',
   'AutoMLJobStatus': 'Completed',
   'AutoMLJobSecondaryStatus': 'Completed',
   'CreationTime': datetime.datetime(2023, 3, 30, 3, 35, 21, 288000, tzinfo=tzlocal()),
   'EndTime': datetime.datetime(2023, 3, 30, 4, 23, 28, 846000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2023, 3, 30, 4, 23, 28, 881000, tzinfo=tzlocal())},
  {'AutoMLJobName': 'automl-dm-18-00-54-46',
   'AutoMLJobArn': 'arn:aws:sagemaker:u

In [29]:
num_existing_jobs = 0
running_jobs = 0

if "AutoMLJobSummaries" in existing_jobs_response.keys():
    job_list = existing_jobs_response["AutoMLJobSummaries"]
    num_existing_jobs = len(job_list)
    # print('[INFO] You already created {} Autopilot job(s) in this account.'.format(num_existing_jobs))
    for j in job_list:
        if "AutoMLJobStatus" in j.keys():
            if j["AutoMLJobStatus"] == "InProgress":
                running_jobs = running_jobs + 1
    print("[INFO] You have {} Autopilot job(s) currently running << Should be 0 jobs.".format(running_jobs))
else:
    print("[OK] Please continue.")

[INFO] You have 1 Autopilot job(s) currently running << Should be 0 jobs.


In [30]:
#Launch Autopilot job

from time import gmtime, strftime, sleep

In [31]:
%store -r auto_ml_job_name_wildfire

try:
    auto_ml_job_name_wildfire
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    auto_ml_job_name_wildfire = "automl-fire-" + timestamp_suffix
    print("Created AutoMLJobName: " + auto_ml_job_name_wildfire)


In [32]:
#job_name = 'automl-fire-03-05-42-40'

#try:
 #   response = sagemaker.describe_auto_ml_job(AutoMLJobName=job_name)
  #  print("The job already exists")
#except:
    #print("The job doesn't exist, creating a new job...")

In [33]:
print(auto_ml_job_name_wildfire)

automl-fire-06-16-52-30


In [34]:
%store auto_ml_job_name_wildfire

Stored 'auto_ml_job_name_wildfire' (str)


In [35]:
max_running_jobs = 1

if running_jobs < max_running_jobs:  # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(
            AutoMLJobName=auto_ml_job_name_wildfire,
            InputDataConfig=input_data_config1,
            OutputDataConfig=output_data_config1,
            AutoMLJobConfig=job_config1,
            RoleArn=role,
        )
        print("[OK] Autopilot Job {} created.".format(auto_ml_job_name_wildfire))
        running_jobs = running_jobs + 1
    except:
        print(
            "[INFO] You have already launched an Autopilot job. Please continue see the output of this job.".format(
                running_jobs
            )
        )
else:
    print(
        "[INFO] You have already launched {} Autopilot running job(s). Please continue see the output of the running job.".format(
            running_jobs
        )
    )

[INFO] You have already launched 1 Autopilot running job(s). Please continue see the output of the running job.


# Analyzing Data and Generate Notebooks

In [36]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)

while (
    "AutoMLJobStatus" not in job_description_response.keys()
    and "AutoMLJobSecondaryStatus" not in job_description_response.keys()
):
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    print("[INFO] Autopilot Job has not yet started. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for Autopilot Job to start...")
    sleep(15)

print("[OK] AutoMLJob started.")

[OK] AutoMLJob started.


# Review the SageMaker `Processing Jobs`
* First Processing Job (Data Splitter) checks the data sanity, performs stratified shuffling and splits the data into training and validation. 
* Second Processing Job (Candidate Generator) first streams through the data to compute statistics for the dataset. Then, uses these statistics to identify the problem type, and possible types of every column-predictor: numeric, categorical, natural language, etc.

In [37]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(
            region
        )
    )
)

In [38]:
%%time

job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]

if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("Starting", "AnalyzingData"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Data analysis phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
[OK] Data analysis phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:8572835

# Generate Notebooks

* Data Exploration
* Candidate Definition

In [39]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)

while "AutoMLJobArtifacts" not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    print("[INFO] Autopilot Job has not yet generated the artifacts. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for AutoMLJobArtifacts...")
    sleep(15)

print("[OK] AutoMLJobArtifacts generated.")

[OK] AutoMLJobArtifacts generated.


In [40]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)

while "DataExplorationNotebookLocation" not in job_description_response["AutoMLJobArtifacts"].keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    print("[INFO] Autopilot Job has not yet generated the notebooks. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for DataExplorationNotebookLocation...")
    sleep(15)

print("[OK] DataExplorationNotebookLocation found.")

[OK] DataExplorationNotebookLocation found.


In [41]:
generated_resources = job_description_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]
download_path = generated_resources.rsplit("/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb")[0]
job_id = download_path.rsplit("/", 1)[-1]

In [42]:
from IPython.core.display import display, HTML

if not job_id:
    print("No AutoMLJobArtifacts found.")
else:
    display(
        HTML(
            '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/{}/sagemaker-automl-candidates/{}/">S3 Generated Resources</a></b>'.format(
                bucket, prefix_model_output, auto_ml_job_name_wildfire, job_id
            )
        )
    )

In [43]:
#Download Notebooks and Code

print(download_path)

s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833


In [44]:
try:
    !aws s3 cp --recursive $download_path .
except:
    print('Could not download the generated resources. Make sure the path is correct.')

download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/generated_module/README.md to generated_module/README.md
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/generated_module/MANIFEST.in to generated_module/MANIFEST.in
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/generated_module/candidate_data_processors/dpp0.py to generated_module/candidate_data_processors/dpp0.py
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/generated_module/candidate_data_processors/dpp4.py to

# Feature Engineering

In [45]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(
            region
        )
    )
)

In [46]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/">Batch Transform Jobs</a></b>'.format(
            region
        )
    )
)

In [47]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("FeatureEngineering"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Feature engineering phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress

# Model Training and Tuning

In [48]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">Hyperparameter Tuning Jobs</a></b>'.format(
            region
        )
    )
)

In [49]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(
            region
        )
    )
)

In [50]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("ModelTuning"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Model tuning phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress 

# Status for completion

In [51]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
job_status = job_description_response["AutoMLJobStatus"]
print(job_status)
if job_status not in ("Stopped", "Failed"):
    while job_status not in ("Completed"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
        job_status = job_description_response["AutoMLJobStatus"]
        print(job_status)
        sleep(10)
    print("[OK] Autopilot Job completed.\n")
else:
    print(job_status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed
[OK] Autopilot Job completed.

CPU times: user 267 ms, sys: 23.8 ms, total: 290 ms
Wall time: 9min 17s


# View All Candidates 

In [52]:
candidates_response = sm.list_candidates_for_auto_ml_job(
    AutoMLJobName=auto_ml_job_name_wildfire, SortBy="FinalObjectiveMetricValue"
)

In [53]:
print(candidates_response.keys())

dict_keys(['Candidates', 'ResponseMetadata'])


In [54]:
while "Candidates" not in candidates_response.keys():
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name_wildfire, SortBy="FinalObjectiveMetricValue"
    )
    print("[INFO] Autopilot Job is generating the Candidates. Please wait.")
    print(json.dumps(candidates_response, indent=4, sort_keys=True, default=str))
    sleep(10)

candidates = candidates_response["Candidates"]
print("[OK] Candidates generated.")

[OK] Candidates generated.


In [55]:
print(candidates[0].keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime', 'CandidateProperties'])


In [56]:
while "CandidateName" not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name_wildfire, SortBy="FinalObjectiveMetricValue"
    )
    candidates = candidates_response["Candidates"]
    print("[INFO] Autopilot Job is generating CandidateName. Please wait. ")
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] CandidateName generated.")

[OK] CandidateName generated.


In [57]:
while "FinalAutoMLJobObjectiveMetric" not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name_wildfire, SortBy="FinalObjectiveMetricValue"
    )
    candidates = candidates_response["Candidates"]
    print("[INFO] Autopilot Job is generating FinalAutoMLJobObjectiveMetric. Please wait. ")
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] FinalAutoMLJobObjectiveMetric generated.")

[OK] FinalAutoMLJobObjectiveMetric generated.


In [58]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str)) #Review results of all candidates

[
    {
        "CandidateName": "automl-fire-06-16-52-30DxewazVSU-003-53b62d8b",
        "CandidateProperties": {
            "CandidateArtifactLocations": {
                "Explainability": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/documentation/explainability/output",
                "ModelInsights": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/documentation/model_monitor/output"
            },
            "CandidateMetrics": [
                {
                    "MetricName": "F1",
                    "Set": "Validation",
                    "StandardMetricName": "F1",
                    "Value": 0.9206100106239319
                },
                {
                    "MetricName": "LogLoss",
                    "Set": "Validation",
                    "StandardMetricName": "LogLoss",
                    "Value": 0.3304300010204315
                },
                {
                    "MetricName": "

In [59]:
for index, candidate in enumerate(candidates):
    print(
        str(index)
        + "  "
        + candidate["CandidateName"]
        + "  "
        + str(candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
    )

0  automl-fire-06-16-52-30DxewazVSU-003-53b62d8b  0.9206100106239319
1  automl-fire-06-16-52-30DxewazVSU-007-07633d82  0.8831499814987183
2  automl-fire-06-16-52-30DxewazVSU-006-6869f033  0.8731399774551392
3  automl-fire-06-16-52-30DxewazVSU-002-5aab0928  0.8409000039100647
4  automl-fire-06-16-52-30DxewazVSU-001-424ad5d9  0.8255500197410583
5  automl-fire-06-16-52-30DxewazVSU-010-98870762  0.7789896726608276
6  automl-fire-06-16-52-30DxewazVSU-009-4bcfe41f  0.7472841739654541
7  automl-fire-06-16-52-30DxewazVSU-004-058818f6  0.6683099865913391
8  automl-fire-06-16-52-30DxewazVSU-005-8dc57559  0.6575199961662292
9  automl-fire-06-16-52-30DxewazVSU-008-d766464b  0.5618748068809509


# Inspect Trials using Experiments API

In [60]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=auto_ml_job_name_wildfire + "-aws-auto-ml-job",
)

df = exp.dataframe()
print(df)

                                   TrialComponentName  \
0   automl-fire-06-16-52-30DxewazVSU-009-4bcfe41f-...   
1   automl-fire-06-16-52-30DxewazVSU-008-d766464b-...   
2   automl-fire-06-16-52-30DxewazVSU-010-98870762-...   
3   automl-fire-06-16-52-30DxewazVSU-005-8dc57559-...   
4   automl-fire-06-16-52-30DxewazVSU-004-058818f6-...   
5   automl-fire-06-16-52-30DxewazVSU-001-424ad5d9-...   
6   automl-fire-06-16-52-30DxewazVSU-006-6869f033-...   
7   automl-fire-06-16-52-30DxewazVSU-007-07633d82-...   
8   automl-fire-06-16-52-30DxewazVSU-002-5aab0928-...   
9   automl-fire-06-16-52-30DxewazVSU-003-53b62d8b-...   
10  automl-fire-06-16-52-30-dpp3-rpb-1-3c006a03602...   
11  automl-fire-06-16-52-30-dpp1-rpb-1-8922150fb9b...   
12  automl-fire-06-16-52-30-dpp2-csv-1-d3b0fa19747...   
13  automl-fire-06-16-52-30-dpp1-1-c1e8245595724c7...   
14  automl-fire-06-16-52-30-dpp3-1-38b6ad346deb448...   
15  automl-fire-06-16-52-30-dpp2-1-13a5d81bcc6d42d...   
16  automl-fire-06-16-52-30-db-

# Explore Best Candidate

In [61]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)

In [62]:
print(best_candidate_response.keys())

dict_keys(['AutoMLJobName', 'AutoMLJobArn', 'InputDataConfig', 'OutputDataConfig', 'RoleArn', 'AutoMLJobConfig', 'CreationTime', 'EndTime', 'LastModifiedTime', 'BestCandidate', 'AutoMLJobStatus', 'AutoMLJobSecondaryStatus', 'GenerateCandidateDefinitionsOnly', 'AutoMLJobArtifacts', 'ResolvedAttributes', 'ResponseMetadata'])


In [63]:
while "BestCandidate" not in best_candidate_response:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    print("[INFO] Autopilot Job is generating BestCandidate. Please wait. ")
    print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate generated.")

[OK] BestCandidate generated.


In [64]:
print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str)) #Results of best candidate model

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-fire-06-16-52-30",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/sagemaker-automl-candidates/automl-fire-06-16-52-30-pr-1-80b34ed57ec1440184060808e2b45ea833/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 10,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-fire-06-16-52-30",
    "AutoMLJobSecondaryStatus": "Completed",
    

In [65]:
print(best_candidate.keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime', 'CandidateProperties'])


In [66]:
while "CandidateName" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate CandidateName generated.")

[OK] BestCandidate CandidateName generated.


In [67]:
while "FinalAutoMLJobObjectiveMetric" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.")

[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.


In [68]:
best_candidate_identifier = best_candidate["CandidateName"]
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"])
print("Metric value: " + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"]))

Candidate name: automl-fire-06-16-52-30DxewazVSU-003-53b62d8b
Metric name: validation:f1_binary
Metric value: 0.9206100106239319


In [69]:
print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

{
    "CandidateName": "automl-fire-06-16-52-30DxewazVSU-003-53b62d8b",
    "CandidateProperties": {
        "CandidateArtifactLocations": {
            "Explainability": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/documentation/explainability/output",
            "ModelInsights": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/documentation/model_monitor/output"
        },
        "CandidateMetrics": [
            {
                "MetricName": "F1",
                "Set": "Validation",
                "StandardMetricName": "F1",
                "Value": 0.9206100106239319
            },
            {
                "MetricName": "LogLoss",
                "Set": "Validation",
                "StandardMetricName": "LogLoss",
                "Value": 0.3304300010204315
            },
            {
                "MetricName": "Recall",
                "Set": "Validation",
                "StandardMetricName": "Recal

# Autopilot Jobs

In [70]:
while "CandidateSteps" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps generated.")

[OK] BestCandidate CandidateSteps generated.


In [71]:
while "CandidateStepType" not in best_candidate["CandidateSteps"][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepType. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps CandidateStepType generated.")

[OK] BestCandidate CandidateSteps CandidateStepType generated.


In [72]:
while "CandidateStepName" not in best_candidate["CandidateSteps"][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepName. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps CandidateStepName generated.")

[OK] BestCandidate CandidateSteps CandidateStepName generated.


In [73]:
best_candidate

{'CandidateName': 'automl-fire-06-16-52-30DxewazVSU-003-53b62d8b',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:f1_binary',
  'Value': 0.9206100106239319,
  'StandardMetricName': 'F1'},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:processing-job/automl-fire-06-16-52-30-db-1-21967a2347fb426a8b3fefdb9b02d68586',
   'CandidateStepName': 'automl-fire-06-16-52-30-db-1-21967a2347fb426a8b3fefdb9b02d68586'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:training-job/automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c',
   'CandidateStepName': 'automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:transform-job/automl-fire-06-16-52-30-dp

In [74]:
steps = []
for step in best_candidate["CandidateSteps"]:
    print("Candidate Step Type: {}".format(step["CandidateStepType"]))
    print("Candidate Step Name: {}".format(step["CandidateStepName"]))
    steps.append(step["CandidateStepName"])

Candidate Step Type: AWS::SageMaker::ProcessingJob
Candidate Step Name: automl-fire-06-16-52-30-db-1-21967a2347fb426a8b3fefdb9b02d68586
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c
Candidate Step Type: AWS::SageMaker::TransformJob
Candidate Step Name: automl-fire-06-16-52-30-dpp3-rpb-1-3c006a03602246a480bc2bb3d2d5
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: automl-fire-06-16-52-30DxewazVSU-003-53b62d8b


In [75]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, steps[0]
        )
    )
)

In [76]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a></b>'.format(
            region, steps[1]
        )
    )
)

In [77]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}">Transform Job</a></b>'.format(
            region, steps[2]
        )
    )
)

In [78]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job (Tuning)</a></b>'.format(
            region, steps[3]
        )
    )
)

In [79]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review All <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(
            region
        )
    )
)

# See Containers and Models Within Inference Pipeline

In [80]:
while "InferenceContainers" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name_wildfire)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate InferenceContainers. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate InferenceContainers generated.")

[OK] BestCandidate InferenceContainers generated.


In [81]:
best_candidate_containers = best_candidate["InferenceContainers"]

In [82]:
for container in best_candidate_containers:
    print(container["Image"])
    print(container["ModelDataUrl"])
    print("======================")

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/data-processor-models/automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/tuning/automl-fir-dpp3-xgb/automl-fire-06-16-52-30DxewazVSU-003-53b62d8b/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/data-processor-models/automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c/output/model.tar.gz


# Update Containers To Show Predicted Label and Confidence Score

In [83]:
for container in best_candidate_containers:
    print(container["Environment"])
    print("======================")

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


In [84]:
best_candidate_containers[1]["Environment"].update({"SAGEMAKER_INFERENCE_OUTPUT": "predicted_label, probability"})
best_candidate_containers[2]["Environment"].update({"SAGEMAKER_INFERENCE_INPUT": "predicted_label, probability"})
best_candidate_containers[2]["Environment"].update({"SAGEMAKER_INFERENCE_OUTPUT": "predicted_label, probability"})

In [85]:
for container in best_candidate_containers:
    print(container["Environment"])
    print("======================")

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


In [86]:
print(best_candidate["InferenceContainers"])

[{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/data-processor-models/automl-fire-06-16-52-30-dpp3-1-38b6ad346deb4484983dac916cb6a74c/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-857283526476/models/autopilot/automl-fire-06-16-52-30/tuning/automl-fir-dpp3-xgb/automl-fire-06-16-52-30DxewazVSU-003-53b62d8b/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predic

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [87]:
%store -r autopilot_model_wf

In [88]:
try:
    autopilot_model_wf
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    autopilot_model_wf = "automl-dm-model-" + timestamp_suffix
    print("[OK] Created Autopilot Model Name: " + autopilot_model_wf)

In [89]:
%store autopilot_model_wf

Stored 'autopilot_model_wf' (str)


In [90]:
%store -r autopilot_model_arn_wf

In [91]:
try:
    autopilot_model_arn_wf
except NameError:
    create_model_response = sm.create_model(
        Containers=best_candidate["InferenceContainers"], ModelName=autopilot_model_wf, ExecutionRoleArn=role
    )
    autopilot_model_arn_wf = create_model_response["ModelArn"]
    print("[OK] Created Autopilot Model: {}".format(autopilot_model_arn_wf))

In [92]:
%store autopilot_model_arn_wf

Stored 'autopilot_model_arn_wf' (str)


# Define EndpointConfig Name

In [93]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
epc_name = "automl-dm-epc-" + timestamp_suffix

print(epc_name)

automl-dm-epc-06-17-33-51


# Define REST Endpoint Name for Autopilot Model

In [94]:
%store -r autopilot_endpoint_wf

In [95]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())

try:
    autopilot_endpoint_wf
except NameError:
    autopilot_endpoint_wf = "automl-dm-ep-" + timestamp_suffix
    print("[OK] Created Autopilot Endpoint Name {}: ".format(autopilot_endpoint_wf))

In [96]:
variant_name = "automl-dm-variant-" + timestamp_suffix
print("[OK] Created Endpoint Variant Name {}: ".format(variant_name))

[OK] Created Endpoint Variant Name automl-dm-variant-06-17-33-51: 


In [97]:
%store autopilot_endpoint_wf

Stored 'autopilot_endpoint_wf' (str)


In [98]:
ep_config = sm.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
            "ModelName": autopilot_model_wf,
            "VariantName": variant_name,
        }
    ],
)

In [99]:
%store -r autopilot_endpoint_arn

In [100]:
try:
    autopilot_endpoint_arn
except NameError:
    create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_wf, EndpointConfigName=epc_name)
    autopilot_endpoint_arn = create_endpoint_response["EndpointArn"]
    print(autopilot_endpoint_arn)

In [101]:
%store autopilot_endpoint_arn

Stored 'autopilot_endpoint_arn' (str)


In [102]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, autopilot_endpoint_wf
        )
    )
)

# Store Variables

In [103]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-30-03-35-20'
auto_ml_job_name_wf                                   -> 'automl-fire-03-05-42-40'
auto_ml_job_name_wildfire                             -> 'automl-fire-06-16-52-30'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:857283526476:endpoint
autopilot_endpoint_wf                                 -> 'automl-dm-ep-30-04-47-49'
autopilot_model_arn_wf                                -> 'arn:aws:sagemaker:us-east-1:857283526476:model/au
autopilot_model_wf                                    -> 'automl-dm-model-30-04-47-47'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-857283526476/data/amazon
autopilot_train_s3_wf                                 -> 's3://sagemaker-us-east-1-857283526476/data/wildfi
autopilot_train_s3_wildfire                           -> 's3://sagemaker-us-east-1-857283526476/data/wildfi
b

# Release

In [104]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [105]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>